<a href="https://colab.research.google.com/github/bes82/Motion_Blur/blob/main/Training_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries, unzip files.

This cell has two jobs. 

1.) Import useful libraries for use during the training.

2.) Unzip all images and arrays that will be used for training.

In [1]:
#@title <- Click here to run code. Double click this text to see code. 
# Import necessary functions.

import pandas as pd
import os as os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Clone Repository And Download Network Architecture and Weights.
print("Cloning Repository...")
! git clone https://github.com/bes82/Motion_Blur
print("Done!")
%cd Motion_Blur

# Defining zip-file names.
from zipfile import ZipFile
folder_Names = ["Flow_Folder.zip","Layered_And_Labeled_Tiles_Resized.zip","Original_Image_Tiles_Resized.zip"]
  
# specifying the zip file name
for file_name in folder_Names:
    
  # opening the zip file in READ mode
  with ZipFile(file_name, 'r') as zip:
      # extracting all the files
      print('Extracting all the files from ' + str(file_name) + " now...")
      zip.extractall()
      print('Done!')
# Define source for original tiles and ground truth masks for training. 


Cloning Repository...
Cloning into 'Motion_Blur'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 68 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (68/68), done.
Done!
/content/Motion_Blur
Extracting all the files from Flow_Folder.zip now...
Done!
Extracting all the files from Layered_And_Labeled_Tiles_Resized.zip now...
Done!
Extracting all the files from Original_Image_Tiles_Resized.zip now...
Done!


## Train the network.

The following cell will train the network using the images unzipped in the previous cell. Each epoch will be displayed dynamically, letting the user see the progress of training. 

The network will stop training after five consecutive epochs with no decrease in the validation loss. 

After training is completed, the network will be saved with the name "Motion_Blur_Network.h5" to the "Motion_Blur" directory.

In [2]:
#@title <- Click here to run code. Double click this text to see code. 
# This chunk of code is used to complete the training of a network which will be used to classify pixels of whole channel
# images.

# Loading useful classes and functions for use in training.
from train_source import DataGenerator
from train_source import Phase1_Net

# Defining paths for a directory of images/masks and also for the data frame of file names.
flow_Directory = "Flow_Folder/"
data_Frame_Location = "Training_Data_Frame_Motion_Blur.csv"

# Read in data frame.
training_Data_Frame = pd.read_csv(data_Frame_Location)

# Defining the framework for how files are named for training.
tile_Names_Style = "Original_Tiles_Resized" 
mask_Names_Style = "Layer_Labeled_Resized"

# Input size for the network.
img_size = (128,128)

indices = np.arange(1500) + 1
np.random.shuffle(indices)

# Number of classes for classification.
num_classes = 2
number_Of_Epochs = 200

# Creating the model.
model = Phase1_Net(img_size, num_classes)

# Creating a callback function for stopping training after no improvement for a set amount of epochs.
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=2,
    mode='auto', baseline=None, restore_best_weights=True
)

# Compiling the network architecture.
model.compile(Adam(lr=0.001),
                 metrics = ['accuracy'],
                 loss = tf.keras.losses.CategoricalCrossentropy())

# Defining the generator which will read in tiles for training in small batches, so we don't use too much RAM.
train_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=True,
                    horizontal_Flips = True,
                    rotations = True,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Training",
                    number_Of_Classes = num_classes,
                         indices = indices)

# Defining the generator which will read in tiles for validation in small batches, so we don't use too much RAM.
validate_Gen = DataGenerator(data_Frame=training_Data_Frame,
                    x_Col = "X",
                    y_Col = "Y_True",
                    directory = flow_Directory,
                    vertical_Flips=False,
                    horizontal_Flips = False,
                    rotations = False,
                    split = True,
                    training_Ratio = 0.8,
                    shuffle = True,
                    tile_Namesake = tile_Names_Style,
                    mask_Namesake = mask_Names_Style,
                    subset = "Validation",
                    number_Of_Classes = num_classes,
                            indices = indices)

# Fitting the hyperparameters of the model.
training_History = model.fit(train_Gen,
                             validation_data = validate_Gen,
                                 epochs=number_Of_Epochs, 
                                 verbose = 1, callbacks = [callback])

# Save the architecture and weights as a .h5 file.
model_Name = "Motion_Blur_Network.h5"
model.save(model_Name)

No errors for filter size:256
No errors for filter size:512
No errors for filter size:512


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
37/37 [==============================] - 830s 22s/step - loss: 0.5180 - accuracy: 0.8674 - val_loss: 0.1460 - val_accuracy: 0.9848
Epoch 2/200
37/37 [==============================] - 822s 22s/step - loss: 0.3059 - accuracy: 0.9864 - val_loss: 0.0628 - val_accuracy: 0.9968
Epoch 3/200
37/37 [==============================] - 821s 22s/step - loss: 0.1726 - accuracy: 0.9886 - val_loss: 0.0403 - val_accuracy: 0.9968
Epoch 4/200
37/37 [==============================] - 820s 22s/step - loss: 0.1021 - accuracy: 0.9886 - val_loss: 0.0282 - val_accuracy: 0.9968
Epoch 5/200
37/37 [==============================] - 824s 22s/step - loss: 0.0689 - accuracy: 0.9885 - val_loss: 0.0224 - val_accuracy: 0.9968
Epoch 6/200
37/37 [==============================] - 822s 22s/step - loss: 0.0516 - accuracy: 0.9885 - val_loss: 0.0181 - val_accuracy: 0.9968
Epoch 7/200
37/37 [==============================] - 822s 22s/step - loss: 0.0411 - accuracy: 0.9884 - val_loss: 0.0133 - val_accuracy: 0.9968